In [1]:
from setup_general import *

# setup data for classifier

In [3]:
data = train_prep.copy()
rebalancing = False

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

# Possible Rebalancing with SMOTE

In [ ]:
if rebalancing == True:
    from imblearn.over_sampling import SMOTE, ADASYN
    from collections import Counter

    def strat(data):
        b = Counter(data).values()
        a = Counter(data).keys()
        a = list(a)
        b = list(b)

        for i in range(len(a)):
            if b[i] < 100 :
                b[i] = 100
        return dict(zip(a, b))

    X_train, y_train = SMOTE(sampling_strategy=strat(y_train), random_state=1).fit_resample(X_train, y_train)


# single rf classifier

In [4]:
rfc = RandomForestClassifier(n_estimators=2000, max_depth=100, max_features=None, random_state=0)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)
probs = rfc.predict_proba(X_test)
val_acc = accuracy_score(y_test, y_pred)

y_pred = rfc.predict(X_train)
probs = rfc.predict_proba(X_train)
train_acc = accuracy_score(y_train, y_pred)

print(train_acc, val_acc)


'\n\ny_pred = rfc.predict(X_test)\nprobs = rfc.predict_proba(X_test)\nval_acc = accuracy_score(y_test, y_pred)\n\ny_pred = rfc.predict(X_train)\nprobs = rfc.predict_proba(X_train)\ntrain_acc = accuracy_score(y_train, y_pred)\n\nprint(train_acc, val_acc)\n'

# sweeping

In [ ]:
project = 'rf'

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'split': {'values': [2, 5, 10, 100]},
        'depth': {'values': [3, 6, 10, 50, 100, 1000]},
        'leaf': {'values': [2, 6, 10, 50, 100, 1000, 5000]},
        'estimators': {'values': [200, 500, 1000, 2000]},
        'features': {'values': [None, 'sqrt', 'log2']},

     }
}

# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

def main():
    run = wandb.init(project=project)

    # note that we define values from `wandb.config` instead 
    # of defining hard values 
    split = wandb.config.split
    depth = wandb.config.depth
    leaf = wandb.config.leaf
    estimators = wandb.config.estimators
    feat = wandb.config.features
    

    # -------------------------- usual training code starts here  -------------------------------------
    
    rfc = RandomForestClassifier(n_estimators=estimators, max_depth=depth, min_samples_leaf=leaf, max_features=feat, min_samples_split=split, random_state=42)
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_test)
    val_acc = accuracy_score(y_test, y_pred)
    
    y_pred = rfc.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)

    print(train_acc, val_acc)

    # -------------------------- ends here  -------------------------------------
    

    wandb.log({
      'train_acc': train_acc,
      'val_acc': val_acc,
    })

# Start sweep job.
wandb.agent(sweep_id, function=main)

# save model

In [5]:
pickle.dump(rfc, open('./models/rf/train_prep_full_best', 'wb'))
loaded_model = pickle.load(open('./models/rf/first_try', 'rb'))

# submission from model

In [ ]:
test_set = test_prep.drop('type', axis=1)
submission = pd.DataFrame({'id': test_set.index ,'type': rfc.predict(test_set)})
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('submissions/some_rf_model.csv', index=False)